In [5]:
from tqdm import trange, tqdm

def generate_predictions(centers, deamon=None, return_set_pred=False, real_mode='test',
                          EPS=0.05, Q_VALUE=0.99):
    preds = {}
    set_preds = {}

    if real_mode == 'test':
        end_point = val_end
        init_point = test_init
    else:
        end_point = train_end
        init_point = val_init
#мы двигаем окно PTS раз, для каждого окна у нас по ключу startpoint предскащания на STEPS шагов вперед 
#то есть в итоге у нас на выходе: {(startpoint, [predicted values for each step])}

    for start_point in tqdm(range(PTS)):
        
        
        preds[start_point] = [None] * (start_point + 1)
        if return_set_pred:
            set_preds[start_point] = [None] * (start_point + 1)

        
        wind = list(map(lambda x: (x, 1), xs[end_point + start_point : init_point + start_point]))

        #Здесь сначала в ряде ставим окно которое из которого семпл, (x,1 ) - здесь 1 - изначальное q-value для 
        #точки ряда, потом перемещая окно у нас уже другое q-value, которое где-то в самом низу считается (2)
        #на каждом шаге первого цикла мы снова его задаем с уже другой точкой изначальной ( двигаем на 1 по x)
        
 
        for step in range(1, STEPS + 1):
        #на каждом шаге этого цикла мы двигаем наше окно, добавляя предсказанную раньше точку вместе с qvalue туда
        #у каждой точки изначального ряда qvalue=1, дальше он уменьшается, видимо это отражает то что с каждым шагом меньше
        #гарантий что норм точка
            x_tests_for_point = {}
            for pattern in patterns:
                
                key = str_subseq(pattern + (WINDOW - 1,)) 
                sample = gen_sample_in_point_with_q(np.concatenate([wind, [(0, 0)]]), 
                                                    WINDOW, pattern, len(wind))
                if not sample:
                    x_tests_for_point[key] = None
                else:
                    x_tests_for_point[key] = sample
                #Здесь короче x_tests_for_point = {} потом туда добавляем сэмпл из окна для каждого паттерна 
                #чтобы потом вес дать каждому изходя из мотива ( здесьэто centers типа центры кластеров)

            chosen_centers = []
            for pattern, centers_values in centers.items():
                if not x_tests_for_point[pattern]:
                    continue
                vector = np.array(x_tests_for_point[pattern][:-1])[:, 0]
                #q-values это какой-то параметр численный который тут сделан 1)) и который юзается для
                #q_weighted prediction типа там он вместе с расстоянием
              
            # def aggr_q(pts):
                    # weighted sum by q-value
                #    if not pts:
                 #   return None
                 #sum_weight = sum(map(lambda center: center[2], pts))
                #pred = sum(map(lambda center: center[0] * center[2], pts))
                 #   return pred / sum_weight
                    
                    
                
                q_values = np.array(x_tests_for_point[pattern][:-1])[:, 1]

                for center in centers_values:
                    dist = euclidean(vector, center[:-1]) #считаем расстояние между векторами для d-weighted предск
                    if dist < EPS:
                        weight_d = (EPS - dist) / EPS
                        weight_q = np.mean(q_values) * Q_VALUE
                        #как я понял сначала у каждой точки которая точно в ряде вес 1, потом он уменьшается 
                        #у каждой новой добавленной точки  тк там значение предсказанное ( неточное -> вес<1)
                        chosen_centers.append((pattern, center, weight_d, weight_q))
                    

            last_points = list(map(lambda center: (center[1][-1], center[2], center[3], center[0]), 
                                  chosen_centers))
            #лист вида[(последняя точка центра кластера, d вес, q вес, pattern)], такой формат хз почему
            
            # deamon predict
            result_point = deamon.predict(start_point, step, last_points)# simple deamon == weighted average of motif-vals
            preds[start_point].append(result_point)
            #тут мы добавляем средневзвешенное значение для точки t+step 
            #( типа предсказание, 2 типа - по расстоянию или qvalue) которые можно посмотреть в классе deamon

            if return_set_pred:
                set_preds[start_point].append(last_pА oints)

            
            if result_point: # мы добавляем пару
                             #(predictedValue, qValue) когда  двигаемся по ряду (2)
                q_value = np.mean(np.array(list(map(lambda center: center[2], 
                                  chosen_centers))))
            else:
                q_value = None
            
            # тут мы двигаем окно дальше добавляя предсказанную точку и дальше ее используя в предсказании следующей
            #точки
            
            wind = np.concatenate([wind[1:], [(result_point, q_value)]])

    deamon.predicted = True

    if return_set_pred:
        return preds, set_preds
    return preds
#возвращаем словарь вида {(точка начала: n предсказанных значений)} -> для каждой точки у нас много разных значений??

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed